This script is used for simulating and obtaining the ratio of the full handshake to the fast handshake with session resumption mechanisms in a simulated environment.

In [2]:
import numpy as np

# Define simulation parameters
N = 10  # number of clients
# N = 20
S = 1  # number of servers
E = 300  # session expiration time (in seconds)
# E = 30
T = 24*60*60  # simulation time (in seconds)
# T = 300
full_handshake_count = 0  # counter for full handshakes
quick_handshake_count = 0  # counter for quick handshakes

rate_600s = 17.361
# Define client and server states
client_states = [{'session': None, 'create_time': None, 'valid': None} for i in range(N)]
# Initialize simulation environment
client_request_times = [np.cumsum(np.random.exponential(600 / rate_600s, sum(np.random.poisson(rate_600s, T//600)))).tolist() for i in range(N)]

# print(client_request_times)

# Define event handling functions
def handle_client_request(client_id, time):
    global full_handshake_count, quick_handshake_count

    # Check if client has an active session
    if client_states[client_id]['session'] is not None:
        # Check if session is still valid
        # if time - client_states[client_id]['create_time'] <= E:
        if client_states[client_id]['valid'] == True:
            # Perform quick handshake
            quick_handshake_count += 1
            return

    # Perform full handshake
    full_handshake_count += 1
    # Create new session
    client_states[client_id]['session'] = 1
    client_states[client_id]['create_time'] = time
    client_states[client_id]['valid'] = True

    return

def handle_timer_update(client_id, time):
    # Check if client has an active session
    if client_states[client_id]['session'] is not None:
        # Check if session has expired
        if time - client_states[client_id]['create_time'] > E:
            client_states[client_id]['valid'] = False
    while len(client_request_times[client_id]) > 0 and client_request_times[client_id][0] <= time:
        handle_client_request(client_id, time)
        client_request_times[client_id].pop(0)

# Run simulation loop
for i in range(0, T, 1):
    # Handle timer update event for all clients
    for j in range(N):
        handle_timer_update(j, i)

total_handshakes = full_handshake_count + quick_handshake_count
full_ratio = full_handshake_count / total_handshakes
quick_ratio = quick_handshake_count / total_handshakes

print("total_handshakes: ", total_handshakes)
print("full_ratio: ", full_ratio)
print("quick_ratio: ", quick_ratio)

total_handshakes:  24756
full_ratio:  0.10373242850218128
quick_ratio:  0.8962675714978187
